# === Dataset Structure ===
 - 43 participants, 3 sessions (days), 17 gestures, 7 trials per gesture
 - Each .mat file contains EMG recordings for one participant/session

In [4]:
from natsort import natsorted
import numpy as np
import scipy.io as sio
from pathlib import Path


# === Step 1: List available .mat files ===
data_path = Path("/Users/jasonwang/Desktop/Wearable/EMG_Project/grabmyo_1.1.0/OutputBM/Session1_converted")
mat_files = natsorted(list(data_path.glob("*.mat")))

print(f"\nFound {len(mat_files)} .mat files:")
for file in mat_files[:5]:
    print(f"  - {file.name}")

# === Step 2: Load and inspect a sample .mat file ===
if mat_files:
    sample_file = mat_files[0]
    print(f"\nExamining file: {sample_file.name}")
    data = sio.loadmat(sample_file)

    print("\nVariables in .mat file:")
    for key, value in data.items():
        if not key.startswith('__'):
            print(f"  - {key}: {type(value).__name__} {getattr(value, 'shape', '')}")

    # === Step 3: Explore EMG data arrays ===
    for key, value in data.items():
        if not key.startswith('__'):
            if isinstance(value, np.ndarray):
                print(f"\n{key} details:")
                print(f"  - Shape: {value.shape}")
                print(f"  - Data type: {value.dtype}")

                if value.dtype == 'object':
                    if value.size > 0:
                        first_element = value.flat[0]
                        print(f"  - First element type: {type(first_element)}")
                        if hasattr(first_element, 'shape'):
                            print(f"  - Shape: {first_element.shape}")
                            print(f"  - Data type: {first_element.dtype}")
                            if hasattr(first_element, 'min'):
                                print(f"  - Min/Max: {first_element.min():.3f} / {first_element.max():.3f}")

                        print("  Shapes of first 3 elements:")
                        for i in range(min(3, value.size)):
                            element = value.flat[i]
                            if hasattr(element, 'shape'):
                                print(f"  - Element {i}: {element.shape}")
                else:
                    print(f"  - Min/Max: {value.min():.3f} / {value.max():.3f}")
                    print(f"  - Sample values: {value.flat[:5]}")



Found 43 .mat files:
  - session1_participant1.mat
  - session1_participant2.mat
  - session1_participant3.mat
  - session1_participant4.mat
  - session1_participant5.mat

Examining file: session1_participant1.mat

Variables in .mat file:
  - DATA_FOREARM: ndarray (7, 17)
  - DATA_WRIST: ndarray (7, 17)

DATA_FOREARM details:
  - Shape: (7, 17)
  - Data type: object
  - First element type: <class 'numpy.ndarray'>
  - Shape: (10240, 16)
  - Data type: float64
  - Min/Max: -0.626 / 0.538
  Shapes of first 3 elements:
  - Element 0: (10240, 16)
  - Element 1: (10240, 16)
  - Element 2: (10240, 16)

DATA_WRIST details:
  - Shape: (7, 17)
  - Data type: object
  - First element type: <class 'numpy.ndarray'>
  - Shape: (10240, 12)
  - Data type: float64
  - Min/Max: -0.316 / 0.252
  Shapes of first 3 elements:
  - Element 0: (10240, 12)
  - Element 1: (10240, 12)
  - Element 2: (10240, 12)


In [9]:
# Quick check for NaNs and Infs in all EMG arrays
for k, v in data.items():
    if not k.startswith('__') and isinstance(v, np.ndarray):
        arrs = v.flat if v.dtype == 'object' else [v]
        nan = inf = total = 0
        for arr in arrs:
            if isinstance(arr, np.ndarray):
                nan += np.isnan(arr).sum()
                inf += np.isinf(arr).sum()
                total += arr.size
        print(f"{k}: NaN={nan}, Inf={inf}, Clean={nan==0 and inf==0}, Total={total}")


DATA_FOREARM: NaN=0, Inf=0, Clean=True, Total=19496960
DATA_WRIST: NaN=0, Inf=0, Clean=True, Total=14622720


## Data Room:
- sampling_rate = 2048Hz
- recording_duration = 5 sec
- n_channels = 16 for forearm and 12 for wrist
- n_gestures = 17 ('123', '123', ..., 'rest')  # You choose these
- trials_per_gesture = 7